# DeepPavlov - an Open-Source Conversational AI Framework 
---

<img align="center" src="https://deeppavlov.ai/docs/_static/ipavlov_logo.png"/>


The open-source conversational AI framework [DeepPavlov](https://deeppavlov.ai/) offers a free and easy-to-use solution to build dialogue systems. DeepPavlov comes with a bunch of predefined components powered by [TensorFlow](https://www.tensorflow.org) and [Keras](https://keras.io) for solving NLP-related problems. The framework allows you to fine-tune hyperparameters and test models. You can check out the models in our [demo](https://demo.ipavlov.ai).

## Installation
We support `Linux` and `Windows` platforms, `Python 3.6` and `Python 3.7`

In [0]:
!pip install -q deeppavlov

## QuickStart

The [DeepPavlov](https://deeppavlov.ai/) NLP pipelines are defined in the separate configuration files under the *config/faq* folder. List of models is available on
[the doc page](http://docs.deeppavlov.ai/en/master/features/overview.html)

When you are decided on the model and a configuration file, there are two ways to use it

* via **Command Line Interface (CLI)**
* via **Python**

### Command Line Interface (CLI)

To get predictions from a model interactively through CLI, run

```bash 
    python -m deeppavlov interact <config_path> [-d]
```

* `-d` downloads required data -- pretrained model files and embeddings
  (optional).

You can train it in the same simple way:

```bash
    python -m deeppavlov train <config_path> [-d]
```

To train on your own data you need to modify dataset reader path in the
[train config doc](http://docs.deeppavlov.ai/en/master/intro/config_description.html#train-config).
The data format is specified in the corresponding model doc page. 

There are even more actions you can perform with configs:

```bash 
    python -m deeppavlov <action> <config_path> [-d]
```

* `<action>` can be
    * `download` to download model's data (same as `-d`),
    * `train` to train the model on the data specified in the config file,
    * `evaluate` to calculate metrics on the same dataset,
    * `interact` to interact via CLI,
    * `riseapi` to run a REST API server (see
    [doc](http://docs.deeppavlov.ai/en/master/integrations/rest_api.html)),
    * `interactbot` to run as a Telegram bot (see
    [doc](http://docs.deeppavlov.ai/en/master/integrations/telegram.html)),
    * `interactmsbot` to run a Miscrosoft Bot Framework server (see
    [doc](http://docs.deeppavlov.ai/en/master/integrations/ms_bot.html)),
    * `predict` to get prediction for samples from *stdin* or from
      *<file_path>* if `-f <file_path>` is specified.
* `<config_path>` specifies path (or name) of model's config file
* `-d` downloads required data


### Python

To get predictions from a model interactively through Python, run

```python 
    from deeppavlov import build_model

    model = build_model(<config_path>, download=True)

    # get predictions for 'input_text1', 'input_text2'
    model(['input_text1', 'input_text2'])
```

* where `download=True` downloads required data from web -- pretrained model
  files and embeddings (optional),
* `<config_path>` is path to the chosen model's config file (e.g.
  `"deeppavlov/configs/ner/ner_ontonotes_bert_mult.json"`) or
  `deeppavlov.configs` attribute (e.g.
  `deeppavlov.configs.ner.ner_ontonotes_bert_mult` without quotation marks).

You can train it in the same simple way:

```python 
    from deeppavlov import train_model 

    model = train_model(<config_path>, download=True)
```

* `download=True` downloads pretrained model, therefore the pretrained
model will be, first, loaded and then train (optional).

Dataset will be downloaded regardless of whether there was ``-d`` flag or
not.

To train on your own data you need to modify dataset reader path in the
[train config doc](http://docs.deeppavlov.ai/en/master/intro/config_description.html#train-config).
The data format is specified in the corresponding model doc page. 

You can also calculate metrics on the dataset specified in your config file:

```python
    from deeppavlov import evaluate_model 

    model = evaluate_model(<config_path>, download=True)
```

## Configuration Files

The config file consists of four main sections: **dataset_reader**, **dataset_iterator**, **chainer**, and **train**. The **dataset_reader** defines the dataset’s location along with the dataset format. After loading, the data is split into the train, validation, and test sets according to the **dataset_iterator** settings. The **chainer** section of the configuration files consists of three subsections. The **in** and **out** sections define an input and an output to the chainer, whereas the **pipe** section defines a pipeline of the required components to interact with the models.

In [0]:
%load https://raw.githubusercontent.com/deepmipt/DeepPavlov/master/deeppavlov/configs/faq/tfidf_logreg_en_faq.json

# BERT-based Models of DeepPavlov

---

<img align="center" src="https://miro.medium.com/max/875/1*DKlqxPVTddoaqaHpOBIB_g.png"/>

The release of [BERT](https://arxiv.org/abs/1810.04805) (Bidirectional Encoder Representations from Transformers) made the year 2018 an inflection point for the Natural Language Processing community. BERT is a transformer-based technique for pretraining language representations, which produces state-of-the-art results across a wide array of NLP tasks. BERT has been uploaded to TensorFlow Hub and offers seamless integration with DeepPavlov. We integrated BERT into three downstream tasks: **text classification**, **named entity recognition** (and sequence tagging in general), and **question answering**. As a result, we achieved substantial improvements in all these tasks.

## BERT for Text Classification

We use the DeepPavlov BERT-based text classification model to the sentiment analysis problem. It involves identifying a writer’s attitude toward a particular topic. The attitude could be *positive*, *negative*, and *neutral*.

In [0]:
!python -m deeppavlov install insults_kaggle_bert

In [0]:
!python -m deeppavlov interact insults_kaggle_bert -d

In [0]:
from deeppavlov import configs, build_model
model = build_model(configs.classifiers.insults_kaggle_bert, download=True)

In [0]:
model(['hey, how are you?', 'You are so dumb!'])

If you want to train model on your data you need to create configuration file and set up **data_path** to folder with *train.csv*, *valid.csv*, *test.csv* and change **MODEL_PATH** where to save trained model.

In [0]:
import json
from pprint import pprint
model_config = json.load(open(configs.classifiers.insults_kaggle_bert))

pprint(model_config['dataset_reader'])
pprint(model_config['metadata']['variables'])

Then, train the model

```bash
    python -m deeppavlov train config_name
```
or in Python
```python
    from deeppavlov import train_model
    model = train_model(model_config)
```

## BERT for Named Entity Recognition

Given a sequence of tokens (words, and possibly punctuation marks), provide a tag from a predefined tag set for each token in the sequence. For example, we want to extract persons' and organizations' names from the text. Then for the input text:

    Yan Goodfellow works for Google Brain

a NER model needs to provide the following sequence of tags:

    B-PER I-PER    O     O   B-ORG  I-ORG

Where *B-* and *I-* prefixes stand for the beginning and inside of the entity, while *O* stands for out of tag or no tag. Markup with the prefix scheme is called *BIO markup*. This markup is introduced for distinguishing of consequent entities with similar types.

In [0]:
!python -m deeppavlov install ner_ontonotes_bert_mult

In [0]:
!python -m deeppavlov interact ner_ontonotes_bert_mult -d

In [0]:
from deeppavlov import configs, build_model
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

In [0]:
ner_model(['Yan Goodfellow works for Google Brain'])

### Multilingual NER

In [0]:
from deeppavlov import configs, build_model
ner_model_ml = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

In [0]:
ner_model_ml([
"Meteorologist Lachlan Stone said the snowfall in Queensland was an unusual occurrence \
  in a state with a sub-tropical to tropical climate.",
"Церемония награждения пройдет 27 октября в развлекательном комплексе Hollywood and \
  Highland Center в Лос-Анджелесе (штат Калифорния, США).", 
"Das Orchester der Philharmonie Poznań widmet sich jetzt bereits zum zweiten \
  Mal der Musik dieses aus Deutschland vertriebenen Komponisten. Waghalter \
  stammte aus einer jüdischen Warschauer Familie."])

## BERT for Question Answering

Context question answering is the task of finding an answer to a question over a given context (e.g, a paragraph from Wikipedia), where the answer to each question is a segment of the context.

In [0]:
!python -m deeppavlov install squad_bert

In [0]:
!python -m deeppavlov interact squad_bert -d

In [0]:
from deeppavlov import build_model, configs
model_qa = build_model(configs.squad.squad_bert, download=True)

### Multilingual QA

In [0]:
from deeppavlov import build_model, configs
model_qa_ml = build_model(configs.squad.squad_bert_multilingual_freezed_emb, download=True)  

In [0]:
context_en = "In meteorology, precipitation is any product of the condensation of atmospheric \
  water vapor that falls under gravity. The main forms of precipitation include drizzle, rain, \
  sleet, snow, graupel, and hail. Precipitation forms as smaller droplets coalesce via collision \
  with other raindrops or ice crystals within a cloud. Short, intense periods of rain in scattered locations \
  are called showers."

context_fr = "Les précipitations désignent tous les météores qui tombent dans une atmosphère \
  et il peut s'agir de solides ou de liquides selon la composition et la température de cette dernière. \
  Ce terme météorologique est le plus souvent au pluriel et désigne sur la Terre les hydrométéores \
  (cristaux de glace ou gouttelettes d'eau) qui, ayant été soumis à des processus de condensation \
  et d'agrégation à l'intérieur des nuages, sont devenus trop lourds pour demeurer en suspension \
  dans l'atmosphère et tombent au sol ou s'évaporent en virga avant de l'atteindre. Par extension, \
  le terme peut également être utilisé pour des phénomènes similaires sur d'autres planètes ou lunes ayant une atmosphère."

model_qa_ml([context_en, context_fr, context_fr], 
      ["Where do water droplets collide with ice crystals to form precipitation?", 
       "Sous quelle forme peut être précipitation?", 
       "Where the term precipitation can be used?"])

# Useful Links
---

[DeepPavlov Repository](https://github.com/deepmipt/DeepPavlov)

[DeepPavlov Demo Page](https://demo.ipavlov.ai)

[DeepPavlov Documentation](https://docs.deeppavlov.ai)

[Our Forum](https://forum.ipavlov.ai)

[Our Medium](https://medium.com/deeppavlov)